In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymongo import MongoClient
import configparser
import logging
import logging.config
sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
sys.path.append('/Users/macico/Dropbox/btc/commons')
os.environ['KULOKO_DIR'] = '/Users/macico/Dropbox/btc/kuloko/kuloko_handler'
os.environ['MONGO_DIR'] = '/Users/macico/Dropbox/btc/commons/mongodb'
os.environ['LOGDIR'] = '/Users/macico/Dropbox/btc/kuloko/log'
KULOKO_DIR=os.environ['KULOKO_DIR'] 
MONGO_DIR=os.environ['MONGO_DIR'] 
LOGDIR=os.environ['LOGDIR'] 

logging.config.fileConfig(os.path.join(KULOKO_DIR,'ini/logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")


In [3]:
os.path.join(MONGO_DIR,'ini/mongo_config.ini')

'/Users/macico/Dropbox/btc/commons/mongodb/ini/mongo_config.ini'

In [4]:
os.path.join(MONGO_DIR,'ini/mongo_config.ini')

'/Users/macico/Dropbox/btc/commons/mongodb/ini/mongo_config.ini'

In [5]:
os.path.join(MONGO_DIR,'ini/mongo_config.ini')

'/Users/macico/Dropbox/btc/commons/mongodb/ini/mongo_config.ini'

In [6]:
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(MONGO_DIR,'ini/mongo_config.ini'), encoding='utf-8')

from mongodb.src.mongo_handler import MongoHandler

In [7]:

import  kuloko_handler.handler.socket_handler as skt_api

private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(KULOKO_DIR,'ini/private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(KULOKO_DIR,'ini/config.ini'), encoding='utf-8')


['/Users/macico/Dropbox/btc/kuloko/kuloko_handler/ini/config.ini']

In [8]:
# mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")
# mongo_db.insert_one({"test":"rrr"})
# for db_data in mongo_db.find():
#     print(db_data)
# mongo_db.delete_all()

In [9]:
mongo_ini.sections()

['LOCAL', 'REMOTE']

In [10]:
mongo_ini['LOCAL']

<Section: LOCAL>

In [11]:
trades = skt_api.Trade(logger, general_config_ini,private_api_ini)
trades.connect(trades.get_url(),'BTC')
trades.subscribe()
mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")

In [19]:
import time
retry_cnt=0
max_retry_cnt=120
print("START")
while True:
    try:
        time.sleep(10)
        data = trades.get()
        print("Data:{0}, Retry_cnt={1}".format(len(data),retry_cnt))
        if len(data)==0:
            retry_cnt += 1
            continue
        insert_json =[ trades.convert_shape(_data,"json") for _data in data ]
        mongo_db.insert_many(insert_json)
        retry_cnt = 0
    except Exception as e:
        retry_cnt += 1
        print("ERROR",e)
        if retry_cnt < max_retry_cnt:
            continue
        print("END",e)
        break


START
Data:0, Retry_cnt=0
Data:0, Retry_cnt=1
Data:0, Retry_cnt=2
Data:0, Retry_cnt=3
Data:0, Retry_cnt=4
Data:0, Retry_cnt=5
Data:0, Retry_cnt=6
Data:0, Retry_cnt=7
Data:0, Retry_cnt=8
Data:0, Retry_cnt=9
Data:0, Retry_cnt=10
Data:0, Retry_cnt=11
Data:0, Retry_cnt=12
Data:0, Retry_cnt=13
Data:0, Retry_cnt=14
Data:0, Retry_cnt=15
Data:0, Retry_cnt=16
Data:0, Retry_cnt=17
Data:0, Retry_cnt=18
Data:0, Retry_cnt=19
Data:0, Retry_cnt=20
Data:0, Retry_cnt=21
Data:0, Retry_cnt=22
Data:0, Retry_cnt=23
Data:0, Retry_cnt=24
Data:0, Retry_cnt=25
Data:0, Retry_cnt=26
Data:0, Retry_cnt=27
Data:0, Retry_cnt=28
Data:0, Retry_cnt=29
Data:0, Retry_cnt=30
Data:0, Retry_cnt=31
Data:0, Retry_cnt=32
Data:0, Retry_cnt=33
Data:0, Retry_cnt=34
Data:0, Retry_cnt=35
Data:0, Retry_cnt=36
Data:0, Retry_cnt=37
Data:0, Retry_cnt=38
Data:0, Retry_cnt=39
Data:0, Retry_cnt=40
Data:0, Retry_cnt=41
Data:0, Retry_cnt=42
Data:0, Retry_cnt=43
Data:0, Retry_cnt=44
Data:0, Retry_cnt=45
Data:0, Retry_cnt=46
Data:0, Retry_cnt

KeyboardInterrupt: 

In [ ]:
trades.disconnect()